## Computing the transformation matrix between Coralie's guiding camera detector (pixel offsets) and the telescope (offsets in azi and ele)  i.e the transformation matrix between telescope movements in azimuth and altitude (az, alt) and the movement of a target on the detector (x, y) 

In [1]:
import numpy as np
from astropy.io import fits
from photutils.detection import DAOStarFinder
from astropy.stats import mad_std
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import os

/home/alberte2/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def compute_transformation_matrix(ref_image, image1, image2, offsets_arcsec, pix_scale, output_dir):
    """
    Compute the transformation matrix between pixel displacements and telescope offsets,
    and save annotated images for star verification.

    Parameters:
    -----------
    ref_image : str
        Path to the reference FITS image.
    image1 : str
        Path to the first FITS image with applied telescope offsets.
    image2 : str
        Path to the second FITS image with applied telescope offsets.
    offsets_arcsec : list of tuples
        Telescope offsets in azimuth and elevation for image1 and image2 (az, alt).
    pix_scale : float
        Pixel scale in arcseconds per pixel.
    output_dir : str
        Directory to save the annotated images.

    Returns:
    --------
    transformation_matrix : np.ndarray
        The 2x2 transformation matrix mapping (x, y) pixel offsets to (az, alt).
    """

    # Step 1: Extract reference fiber coordinates from ref_image header
    with fits.open(ref_image) as ref_hdul:
        ref_header = ref_hdul[0].header
        ref_x = ref_header['HIERARCH FIBER XREFCUR']
        ref_y = ref_header['HIERARCH FIBER YREFCUR']
        ref_coords = np.array([ref_x, ref_y])

    # Step 2: Detect star centroids in image1 and image2
    def detect_and_save_star(image_path):
        with fits.open(image_path) as hdul:
            data = hdul[0].data
        # Estimate background and detect the star
        bkg = np.median(data)
        mad = mad_std(data - bkg)
        daofind = DAOStarFinder(threshold=5 * mad, fwhm=3.0)  # Adjust FWHM if needed
        sources = daofind(data - bkg)
        if sources is None or len(sources) == 0:
            raise ValueError(f"No stars detected in {image_path}")
        # Use the brightest star
        brightest_star = sources[np.argmax(sources['flux'])]
        x_centroid, y_centroid = brightest_star['xcentroid'], brightest_star['ycentroid']

        # Save annotated image with a circle over the detected star
        fig, ax = plt.subplots()
        im = ax.imshow(data, cmap= 'gray', origin='lower', vmin=bkg, vmax=bkg + 5 * mad)
        circle = Circle((x_centroid, y_centroid), radius=30, color='red', fill=False, label='Detected Star')
        ax.add_patch(circle)
        plt.colorbar(im , ax=ax)
        plt.title(f"Star Detection: {os.path.basename(image_path)}")
        plt.xlabel("X Pixel")
        plt.ylabel("Y Pixel")
        output_filename = os.path.join(output_dir, os.path.basename(image_path).replace('.fits', '.png'))
        plt.savefig(output_filename)
        plt.close()
        print(f"Saved annotated image: {output_filename}")
        return np.array([x_centroid, y_centroid])

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    image1_coords = detect_and_save_star(image1)
    image2_coords = detect_and_save_star(image2)

    # Print coordinates
    print(f"Image 1 Star Coordinates (x, y): {image1_coords}")
    print(f"Image 2 Star Coordinates (x, y): {image2_coords}")

    # Step 3: Compute pixel offsets
    delta_px_1 = image1_coords - ref_coords  # From ref_image to image1
    delta_px_2 = image2_coords - image1_coords  # From image1 to image2

    # Step 4: Convert pixel offsets to arcseconds
    delta_arcsec = np.array([delta_px_1, delta_px_2]) * pix_scale

    # Step 5: Solve for the transformation matrix (delta_arcsec -> offsets_arcsec)
    input_offsets = np.array(offsets_arcsec)  # Telescope offsets in (az, alt)
    transformation_matrix, _, _, _ = np.linalg.lstsq(delta_arcsec, input_offsets, rcond=None)

    return transformation_matrix

In [11]:
# Example usage:
# File paths for reference image and images with applied offsets
ref_image = "/home/alberte2/Sample_docs/Compute_transformation_matrix_coralie/Albert6.fits"  # Replace with actual file path
image1 = "/home/alberte2/Sample_docs/Compute_transformation_matrix_coralie/Albert7.fits"  # With (50, 0) offset
image2 = "/home/alberte2/Sample_docs/Compute_transformation_matrix_coralie/Albert8.fits"  # With (0, 50) offset

# Telescope offsets for image1 and image2 in (az, alt) arcseconds
offsets_arcsec = [(50, 0), (0, 50)]
# Pixel scale in arcseconds per pixel
pix_scale = 0.2575  
# Output directory for annotated images
output_dir = "/home/alberte2/Sample_docs/Compute_transformation_matrix_coralie/"
# Compute the transformation matrix
matrix = compute_transformation_matrix(ref_image, image1, image2, offsets_arcsec, pix_scale, output_dir)
print("Transformation Matrix (x, y -> az, alt):")
print(matrix)

Saved annotated image: /home/alberte2/Sample_docs/Compute_transformation_matrix_coralie/Albert7.png
Saved annotated image: /home/alberte2/Sample_docs/Compute_transformation_matrix_coralie/Albert8.png
Image 1 Star Coordinates (x, y): [651.57569671 572.08968503]
Image 2 Star Coordinates (x, y): [841.68672663 570.6179482 ]
Transformation Matrix (x, y -> az, alt):
[[0.00792989 1.02141226]
 [1.02434018 0.00474254]]
